In [1]:
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from scipy.spatial import distance as dist

import time
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import dlib
from imutils import face_utils
from imutils.video import VideoStream

from threading import Thread

Using Theano backend.


### Locate the face


In [2]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# detect the face rectangle 
def detect(img, cascade = face_cascade , minimumFeatureSize=(20, 20)):
    if cascade.empty():
        raise (Exception('There was a problem loading your Haar Cascade xml file.'))
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=1, minSize=minimumFeatureSize)
 
    # if it doesn’t return rectangle, return array with zero length
    if len(rects) == 0:
        return []
    # detect mutliscale returns (x,y,w,h)
    # x,y are the coordinates of the lower left corner
    # w, h are the width and height of the rectangle
    # convert last coord from (width,height) to (maxX, maxY)
    rects[:, 2:] += rects[:, :2]
    return rects

In [ ]:
def detect(img, face_detector):
    

### Locate the eyes

In [3]:
detector = dlib.get_frontal_face_detector
predictor = dlib.shape_predictor('facial-landmarks/shape_predictor_68_face_landmarks.dat')
def crop_eyes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect the face at grayscale image
    rect = detect(gray, minimumFeatureSize=(100, 100))
    
    # if the face detector doesn’t detect face
    # return None, else if detects more than one faces
    # keep the bigger and if it is only one keep one dim
    if len(rect) == 0:
        return None
    elif len(rect) > 1:
        face = rect[0]
    elif len(rect) == 1:
        [face] = rect
        
    # keep the face region from the whole frame
    face_rect = dlib.rectangle(left = int(face[0]), top = int(face[1]),
                               right = int(face[2]), bottom = int(face[3]))
    
    # determine the facial landmarks for the face region
    shape = predictor(gray, face_rect)
    shape = face_utils.shape_to_np(shape)
            
    # grab the indexes of the facial landmarks for the left and right eye, respectively
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
    
    # extract the left and right eye coordinates
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    
    # compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    left_eye_hull = cv2.convexHull(leftEye)
    right_eye_hull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)
    cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
        
    # keep the upper and the lower limit of the eye and compute the height
    l_uppery = min(leftEye[1:3,1])
    l_lowy = max(leftEye[4:,1])
    l_dify = abs(l_uppery - l_lowy)
    
    # compute the width of the eye
    lw = (leftEye[3][0] - leftEye[0][0])
    
    # take a snapshot of the eye with 100x100 box to be reshaped later
    minxl = (leftEye[0][0] - ((100-lw)/2))
    maxxl = (leftEye[3][0] + ((100-lw)/2))
    minyl = (l_uppery - ((100-l_dify)/2))
    maxyl = (l_lowy + ((100-l_dify)/2))
    
    # crop the eye rectangle from the frame
    left_eye_rect = np.rint([minxl, minyl, maxxl, maxyl])
    left_eye_rect = left_eye_rect.astype(int)
    left_eye_image = gray[(left_eye_rect[1]):left_eye_rect[3], (left_eye_rect[0]):left_eye_rect[2]]
    
    # same process for right eye
    r_uppery = min(rightEye[1:3,1])
    r_lowy = max(rightEye[4:,1])
    r_dify = abs(r_uppery - r_lowy)
    rw = (rightEye[3][0] - rightEye[0][0])
    minxr = (rightEye[0][0]-((100-rw)/2))
    maxxr = (rightEye[3][0] + ((100-rw)/2))
    minyr = (r_uppery - ((100-r_dify)/2))
    maxyr = (r_lowy + ((100-r_dify)/2))
    right_eye_rect = np.rint([minxr, minyr, maxxr, maxyr])
    right_eye_rect = right_eye_rect.astype(int)
    right_eye_image = gray[right_eye_rect[1]:right_eye_rect[3], right_eye_rect[0]:right_eye_rect[2]]
    
    # if it doesn’t detect left or right eye return None
    if 0 in left_eye_image.shape or 0 in right_eye_image.shape:
        return None
    
    # resize for the conv net
    left_eye_image = cv2.resize(left_eye_image, (24, 24))
    right_eye_image = cv2.resize(right_eye_image, (24, 24))
    right_eye_image = cv2.flip(right_eye_image, 1)
    # return left and right eye
    return left_eye_image, right_eye_image



In [4]:
def cnn_preprocess(img):
    img = img.astype('float32')
    # scale between 0 and 1
    img /= 255
    # keras needs (row, width, height, channel)
    # add two dimensions for number of images (row) and channel
    img = np.expand_dims(img, axis=2)
    img = np.expand_dims(img, axis=0)
    return img

In [5]:
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    # load model architecture
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weight_path)
    # compile the loaded model
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    return model

In [6]:
def sound_alarm(path):
    # play an alarm sound
    playsound.playsound(path)

In [7]:
def main():
    counter = 0
    alarm_on = False
    max_frame = 20
    
    # open the camera,load the cnn model
    vs = VideoStream(src=0).start()
    model = load_model('model.json', 'weights.h5')
    time.sleep(2.0)
    
    while True:
        frame = vs.read()
        # detect eyes
        eyes = crop_eyes(frame)
        if eyes is None:
            continue
        else:
            left_eye, right_eye = eyes  
        
        # predict if the eye is open or closed
        state_left = model.predict_classes(cnn_preprocess(left_eye))[0][0]
        state_right = model.predict_classes(cnn_preprocess(right_eye))[0][0]
        
        if state_left == 0 and state_right == 0:
            counter += 1
            cv2.putText(frame, "Closed", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, "For: {:.2f}".format(counter), (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
            # sound alarm if closed too long 
            if counter >= max_frame:
                # if the alarm is not on, turn it on
                if not alarm_on:
                    alarm_on = True
                    t = Thread(target=sound_alarm,
                                args=('siren.wav',))
                    t.deamon = True
                    t.start()

                # draw an alarm on the frame
                cv2.putText(frame, "YOU'RE GOING TO CRASH!", (100, 300),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        else:
            counter = 0
            alarm_on = False
            cv2.putText(frame, "Opened", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        # show the frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
    
    cv2.destroyAllWindows()
    vs.stop()       
    
    

In [8]:
main()

/Users/brendonhapp/anaconda/envs/py36/lib/python3.6/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/brendonhapp/anaconda/envs/py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/brendonhapp/anaconda/envs/py36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-10befc4b31cf>", line 3, in sound_alarm
    playsound.playsound(path)
NameError: name 'playsound' is not defined

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/brendonhapp/anaconda/envs/py36/lib/python3.6/threading.py"